# Toric Decoder
Apply the [2D decoder](https://arxiv.org/abs/1406.2338), followed by [minimum-weight perfect matching](https://arxiv.org/abs/2303.15933), on the toric code with side length $L$. We consider i.i.d. $X$ errors on the qubits (bonds), and work with $m$ anyons (plaquettes). The errors are plotted on the dual lattice by default.
## 2D Decoder
At each time step out of $T=L$ total, perform the following steps:
1. Make an $X$ error on each qubit with probability $p=0.05$.
2. Perform $c$ field updates, according to the following rule:$$\phi\mapsto\phi+\frac\eta4\nabla^2\phi+q$$Where $\phi$ is the auxillary field, $\eta=0.1$ is the Jacobi smoothing parameter, and $q$ is the anyon density with unit mass $-\frac4\eta$. Take $4\pi G=1$, so that $\nabla^2\phi=\rho$.
3. For each anyon, move it its highest-$\phi$ neighboring cell with probability $\frac12$.

In [6]:
# %matplotlib widget

import numpy as np
import cupy as cp
mempool = cp.get_default_memory_pool()
import subprocess as sp
import os
# from tqdm.notebook import tqdm

from toric import State, pcm, mwpm, logical_error
from toric import decoder_2D
from pymatching import Matching

# import matplotlib.pyplot as plt
# plt.rcParams['figure.dpi'] = 150

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.used --format=csv"
    memory_used_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_used_values = [int(x.split()[0]) for x in memory_used_info]
    return memory_used_values[0]

memory_baseline = get_gpu_memory()
print(memory_baseline)

306


In [7]:
# Simulation parameters

N = 1000 # Shots
L = 100 # Lattice size
p_error = 0.004 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 16 # "Field velocity" - number of field updates per cycle
T = L # Epochs

matching = Matching(pcm(L))

In [8]:
state = State(N, L)

In [9]:
expected = np.sum([a.nbytes for a in state]) / 1024**2
reported = mempool.total_bytes() / 1024**2
actual = get_gpu_memory() - memory_baseline
print(f"Memory usage: {actual:.2f} MiB (expected {expected:.2f} MiB, reported {reported:.2f} MiB)")

Memory usage: 40.00 MiB (expected 66.76 MiB, reported 181.20 MiB)


In [10]:
decoder_2D(state, T, c, η, p_error)

OutOfMemoryError: Out of memory allocating 80,000,000 bytes (allocated so far: 190,002,176 bytes, limit set to: 209,715,200 bytes).

In [ ]:
expected = np.sum([a.nbytes for a in state]) / 1024**2
reported = mempool.total_bytes() / 1024**2
actual = get_gpu_memory() - memory_baseline
print(f"Memory usage: {actual:.2f} MiB (expected {expected:.2f} MiB, reported {reported:.2f} MiB)")

Memory usage: 582.00 MiB (expected 66.76 MiB, reported 572.21 MiB)


In [ ]:
x_correction, y_correction = mwpm(matching, state.q)
logical_error(x_correction ^ state.x_error, y_correction ^ state.y_error).mean()

array(0.)

In [ ]:
expected = np.sum([a.nbytes for a in state])*9/7 / 1024**2
reported = mempool.total_bytes() / 1024**2
actual = get_gpu_memory() - memory_baseline
print(f"Memory usage: {actual:.2f} MiB (expected {expected:.2f} MiB, reported {reported:.2f} MiB)")

Memory usage: 582.00 MiB (expected 85.83 MiB, reported 572.21 MiB)


In [ ]:
mempool.free_all_blocks()

In [ ]:
expected = np.sum([a.nbytes for a in state])*9/7 / 1024**2
reported = mempool.total_bytes() / 1024**2
actual = get_gpu_memory() - memory_baseline
print(f"Memory usage: {actual:.2f} MiB (expected {expected:.2f} MiB, reported {reported:.2f} MiB)")

Memory usage: 160.00 MiB (expected 85.83 MiB, reported 152.59 MiB)
